<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>Engenharia de Software Para Machine Learning</font>
## <font color='blue'>Projeto 1</font>
## <font color='blue'>Construção de Aplicação Web e Integração com Machine Learning</font>

In [14]:
!pip install -q -U watermark
!pip install scikit-learn
!pip install pandas
!pip install matplotlib
!pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/294.9 kB ? eta -:--:--
   ------------ --------------------------- 92.2/294.9 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 3.6 MB/s eta 0:00:00


In [15]:
# Imports
import sklearn
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [16]:
%reload_ext watermark
%watermark -a "Mateus Marques"

Author: Data Science Academy



## Carregando os Dados

In [4]:
# Carrega o dataset
df_dsa = pd.read_csv("dataset.csv")

NameError: name 'pd' is not defined

In [ ]:
# Shape
df_dsa.shape

In [ ]:
# Visualiza amostra dos dados
df_dsa.head()

## Engenharia de Atributos e Limpeza de Dados

Nosso objetivo é prever o salário de um profissional de tecnologia. Vamos trabalhar com algumas variáveis preditoras e a variável alvo será ConvertedComp (compensação financeira).

In [ ]:
# Vamos trabalhar somente com algumas variáveis
df = df_dsa[["Country", "DevType", "EdLevel", "YearsCodePro", "Employment", "ConvertedComp"]]

In [ ]:
# Ajustamos o nome de uma das variáveis
df = df.rename({"ConvertedComp": "Salary"}, axis = 1)

In [ ]:
# Amostra dos dados
df.head()

In [ ]:
# Amostra dos dados
df.tail()

In [ ]:
# Info
df.info()

In [ ]:
# Filtramos os dados mantendo somente valores não nulos
df = df[df["Salary"].notnull()]

In [ ]:
# Algumas variáveis apresentam valores ausentes
df.info()

In [ ]:
# Eliminamos registros com valores ausentes
df = df.dropna()

In [ ]:
# Problema resolvido
df.isnull().sum()

In [ ]:
# Valores únicos da variável categórica
df['Employment'].unique()

In [ ]:
# Conta os registros por categoria
df['Employment'].value_counts()

In [ ]:
# Vamos manter somente os registros de quem trabalha full-time
df = df[df["Employment"] == "Employed full-time"]

In [ ]:
df.head()

In [ ]:
# Não precisamos mais desta variável pois agora ela é de fato uma constante
df = df.drop("Employment", axis=1)

In [ ]:
df.info()

In [ ]:
# Valores únicos da variável
df['DevType'].unique()

In [ ]:
# Vamos converter a variável em uma lista fazendo split por ;
df['DevType'] = df['DevType'].str.split(";")

In [ ]:
# Buscamos o primeiro elemento da lista
df['DevType'] = df['DevType'].str[0]

In [ ]:
# Valores únicos
len(df['DevType'].unique())

In [ ]:
# Contagem de registros por categoria
df['DevType'].value_counts()

In [ ]:
# Definição da função que recebe duas entradas: 'categories', que é uma série de dados categóricos, 
# e 'cutoff', que é um limite numérico
def dsa_reduz_categorias(categories, cutoff):
    
    # Inicializa um dicionário vazio para mapear as categorias
    categorical_map = {}
    
    # Inicia um loop que percorre todos os elementos da série 'categories'
    for i in range(len(categories)):
        
        # Verifica se o valor da categoria atual é maior ou igual ao limite (cutoff).
        if categories.values[i] >= cutoff:
            
            # Se verdadeiro, mapeia a categoria para ela mesma no dicionário.
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            
            # Se falso, mapeia a categoria para 'Other'.
            categorical_map[categories.index[i]] = 'Other'
    
    # Após o loop, retorna o dicionário com o mapeamento das categorias.
    return categorical_map

In [ ]:
# Aplica a função filtrando categorias com mais de 100 registros
dev_map = dsa_reduz_categorias(df['DevType'].value_counts(), 100)

In [ ]:
# Agora mapeamos o item anterior ao valor atual da variável e salvamos na própria variável
df['DevType'] = df['DevType'].map(dev_map)

In [ ]:
# Registros por categoria
df['DevType'].value_counts()

In [ ]:
# Registros por categoria
df['Country'].value_counts()

In [ ]:
# Criamos o mapeamento filtrando categorias com mais 400 registros
country_map = dsa_reduz_categorias(df.Country.value_counts(), 400)

In [ ]:
# Aplicamos o mapeamento
df['Country'] = df['Country'].map(country_map)

In [ ]:
# Registros por categoria
df.Country.value_counts()

In [ ]:
# Plot
fig, ax = plt.subplots(1,1, figsize = (12, 7))
df.boxplot('Salary', 'Country', ax = ax)
plt.suptitle('Salário Por País')
plt.title('')
plt.ylabel('Salary')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Vamos aplicar mais um filtro aos dados removendo outliers e categorias do tipo "Outros"
df = df[df['Salary'] <= 250000]
df = df[df['Salary'] >= 5000]
df = df[df['Country'] != 'Other']
df = df[df['DevType'] != 'Other']
df = df[df['DevType'] != 'Other (please specify):']

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Plot
fig, ax = plt.subplots(1,1, figsize = (12, 7))
df.boxplot('Salary', 'Country', ax = ax)
plt.suptitle('Salário Por País')
plt.title('')
plt.ylabel('Salary')
plt.xticks(rotation = 45)
plt.show()

In [ ]:
# Valores únicos
df["YearsCodePro"].unique()

In [ ]:
# Função para converter os valores categóricos identificados acima para valores numéricos
def dsa_ajusta_tempo_experiencia(x):
    if x ==  'More than 50 years':
        return 50
    if x == 'Less than 1 year':
        return 0.5
    return float(x)

In [ ]:
# Aplica a função
df['YearsCodePro'] = df['YearsCodePro'].apply(dsa_ajusta_tempo_experiencia)

In [ ]:
# Valores únicos
df["YearsCodePro"].unique()

In [ ]:
# Valores únicos
df["EdLevel"].unique()

In [ ]:
# Função para reduzir o número de categorias da variável
def dsa_ajusta_nivel_educacional(x):
    if 'Bachelor’s degree' in x:
        return 'Bachelor’s degree'
    if 'Master’s degree' in x:
        return 'Master’s degree'
    if 'Professional degree' in x or 'Other doctoral' in x:
        return 'Post grad'
    return 'Less than a Bachelors'

In [ ]:
# Aplica a função
df['EdLevel'] = df['EdLevel'].apply(dsa_ajusta_nivel_educacional)

In [ ]:
# Registros por categoria
df["EdLevel"].value_counts()

In [ ]:
# Verifique periodicamente se valores ausentes surgiram durante as transformações
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# Mapeamento para converter a variável categórica em representação numérica
map_ed_level = {'Less than a Bachelors': 0, 
                'Bachelor’s degree': 1, 
                'Master’s degree': 2, 
                'Post grad': 3}

In [ ]:
# Aplica o mapeamento
df['EdLevel'] = df['EdLevel'].map(map_ed_level)

In [ ]:
# Mapeamento para converter a variável categórica em representação numérica
map_pais = {'Other': 0,  
            'United States': 1,
            'India': 2,
            'United Kingdom': 3,
            'Germany': 4,
            'Canada': 5,
            'Brazil': 6,
            'France': 7,
            'Spain': 8,
            'Australia': 9,
            'Netherlands': 10,
            'Poland': 11,
            'Italy': 12,
            'Russian Federation': 13,
            'Sweden': 14 }

In [ ]:
# Aplica o mapeamento
df['Country'] = df['Country'].map(map_pais)

In [ ]:
# Mapeamento para converter a variável categórica em representação numérica
map_dev_type = {'Developer, back-end': 0,
                'Developer, full-stack': 1,
                'Database administrator': 2,
                'Developer, front-end': 3, 
                'Data or business analyst': 4,
                'Academic researcher': 5,
                'Designer': 6,
                'Developer, desktop or enterprise applications': 7,
                'Data scientist or machine learning specialist': 8,
                'Developer, mobile': 9,
                'Developer, embedded applications or devices': 10,
                'Other': 11,
                'DevOps specialist': 12,
                'Developer, QA or test': 13,
                'Engineer, data': 14,
                'Engineering manager': 15,
                'Developer, game or graphics': 16}

In [ ]:
# Aplica o mapeamento
df['DevType'] = df['DevType'].map(map_dev_type)

In [ ]:
# Verifique periodicamente se valores ausentes surgiram durante as transformações
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# Mapa de Correlação
plt.figure(figsize = (8,6))
corr = df.corr()
sns.heatmap(corr, annot = True);

## Pré-Processamento dos Dados

In [ ]:
# Separamos X e y
X = df.drop("Salary", axis = 1)
y = df["Salary"]

In [ ]:
# Divisão em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# print the shape of the training and testing datasets
print(f"Shape de X_treino :{X_treino.shape}\nShape de X_teste: {X_teste.shape}",
      f"\nShape de y_treino: {y_treino.shape}\nShape de y_teste: {y_teste.shape}")

In [ ]:
# Cria o padronizador
scaler = StandardScaler()

In [ ]:
# Fit e transform somente em treino
X_treino_scaled = scaler.fit_transform(X_treino)

In [ ]:
# Transform em teste (e em novos dados)
X_teste_scaled = scaler.transform(X_teste)

In [ ]:
# Salva o padronizador pois será usado na app
pickle.dump(scaler, open('dsa_scaler.pkl','wb'))

In [ ]:
X_treino_scaled

In [ ]:
X_teste_scaled

In [ ]:
# Cria uma figura e um conjunto de subplots.
fig, ax = plt.subplots(1, 2, figsize = (15, 5))

# Primeiro subplot para X_train antes da padronização
sns.boxplot(data = X_treino, ax = ax[0])
ax[0].set_title('X_treino Antes da Padronização')

# Segundo subplot para X_treino após a padronização
sns.boxplot(data = X_treino_scaled, ax = ax[1])
ax[1].set_title('X_treino Depois da Padronização')

plt.show()

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
## Modelagem Preditiva

### Versão 1 do Modelo

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
# Cria o modelo
modelo_dsa_v1 = LinearRegression()

In [ ]:
# Treina o modelo
modelo_dsa_v1.fit(X_treino_scaled, y_treino)

In [ ]:
# Coeficientes aprendidos durante o treinamento
print(modelo_dsa_v1.coef_)
print(modelo_dsa_v1.intercept_)

In [ ]:
# Hiperparâmetros do modelo
modelo_dsa_v1.get_params()

In [ ]:
# Faz previsões
y_pred_v1 = modelo_dsa_v1.predict(X_teste_scaled)

In [ ]:
# Métricas
print('Mean Absolute Error (MAE):', round(mean_absolute_error(y_teste, y_pred_v1),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(mean_squared_error(y_teste, y_pred_v1)),3))
print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(mean_squared_error(y_teste, y_pred_v1))),3))
print('R2 Score:', round(r2_score(y_teste, y_pred_v1),6))

### Versão 2 do Modelo

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

In [ ]:
# Cria o modelo
modelo_dsa_v2 = DecisionTreeRegressor(random_state = 1)

In [ ]:
# Treina o modelo
modelo_dsa_v2.fit(X_treino_scaled, y_treino)

In [ ]:
# Faz previsões
y_pred_v2 = modelo_dsa_v2.predict(X_teste_scaled)

In [ ]:
# Métricas
print('Mean Absolute Error (MAE):', round(mean_absolute_error(y_teste, y_pred_v2),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(mean_squared_error(y_teste, y_pred_v2)),3))
print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(mean_squared_error(y_teste, y_pred_v2))),3))
print('R2 Score:', round(r2_score(y_teste, y_pred_v2),6))

### Versão 3 do Modelo

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
# Cria o modelo
modelo_dsa_v3 = RandomForestRegressor(random_state = 1)

In [ ]:
# Treina o modelo
modelo_dsa_v3.fit(X_treino_scaled, y_treino)

In [ ]:
# Faz previsões
y_pred_v3 = modelo_dsa_v3.predict(X_teste_scaled)

In [ ]:
# Métricas
print('Mean Absolute Error (MAE):', round(mean_absolute_error(y_teste, y_pred_v3),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(mean_squared_error(y_teste, y_pred_v3)),3))
print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(mean_squared_error(y_teste, y_pred_v3))),3))
print('R2 Score:', round(r2_score(y_teste, y_pred_v3),6))

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
### Otimização de Hiperparâmetros

In [ ]:
# Grid de hiperparâmetros para otimizar o modelo
max_depth = [None,6,8,10,12]
n_estimators = [100,200,300]
min_samples_leaf = [2,3,4,5]
parameters = {"max_depth": max_depth, "n_estimators": n_estimators, "min_samples_leaf": min_samples_leaf}

In [ ]:
# Cria o modelo
modelo_dsa = RandomForestRegressor(random_state = 1)

In [ ]:
# Cria o Grid Search
grid_search_dsa = GridSearchCV(modelo_dsa, parameters, scoring = 'neg_mean_squared_error')

Treinamos o Grid Search buscando a melhor combinação de hiperparâmetros.

In [ ]:
%%time
grid_search_dsa.fit(X_treino_scaled, y_treino)

In [ ]:
# O modelo final é o melhor estimador encontrado pelo Grid Search
modelo_dsa = grid_search_dsa.best_estimator_

In [ ]:
# Treina o melhor estimador
modelo_dsa.fit(X_treino_scaled, y_treino)

In [ ]:
# Faz previsões
y_pred = modelo_dsa.predict(X_teste_scaled)

In [ ]:
# Métricas
print('Mean Absolute Error (MAE):', round(mean_absolute_error(y_teste, y_pred),3))  
print('Root Mean Squared Error (RMSE):', round(np.sqrt(mean_squared_error(y_teste, y_pred)),3))
print('Root Mean Squared Log Error (RMSLE):', round(np.log(np.sqrt(mean_squared_error(y_teste, y_pred))),3))
print('R2 Score:', round(r2_score(y_teste, y_pred),6))

## Testando o Modelo Antes do Deploy

In [ ]:
# Vamos gerar novos dados para testar o modelo antes do deploy
novos_dados = np.array([["United States", 'Master’s degree', 'Developer, full-stack', 15]])
novos_dados

In [ ]:
# Extrai o mapeamento
country_mapping = {country: i for i, country in enumerate(np.unique(novos_dados[:, 0]))}
education_mapping = {education: i for i, education in enumerate(np.unique(novos_dados[:, 1]))}
devtype_mapping = {devtype: i for i, devtype in enumerate(np.unique(novos_dados[:, 2]))}

In [ ]:
# Vetoriza X após aplicar o mapeamento às variáveis categóricas
novos_dados[:, 0] = np.vectorize(country_mapping.get)(novos_dados[:, 0])
novos_dados[:, 1] = np.vectorize(education_mapping.get)(novos_dados[:, 1])
novos_dados[:, 2] = np.vectorize(devtype_mapping.get)(novos_dados[:, 2])

In [ ]:
# Converte a matriz para float a fim de aumentar a precisão
novos_dados = novos_dados.astype(float)

In [ ]:
print(novos_dados)

In [ ]:
# Transform em novos dados
novos_dados_scaled = scaler.transform(novos_dados)

In [ ]:
print(novos_dados_scaled)

In [ ]:
# Previsão do salário com o modelo
y_pred = modelo_dsa.predict(novos_dados_scaled)

In [ ]:
print(y_pred)

In [ ]:
# Modelo final para salvar em disco e seguir para o deploy
# Salvamos também o mapeamento
modelo_dsa_final = {"model": modelo_dsa, 
                    "country_mapping": country_mapping, 
                    "education_mapping": education_mapping, 
                    "devtype_mapping":devtype_mapping}

In [ ]:
# Salva o modelo em disco
with open('dsa_modelo.pkl', 'wb') as file:
    pickle.dump(modelo_dsa_final, file)

In [ ]:
# Carrega o modelo para verificar se está funcionando
with open('dsa_modelo.pkl', 'rb') as file:
    modelo_versao_final = pickle.load(file)

In [ ]:
# Carrega o padronizador para verificar se está funcionando
with open('dsa_scaler.pkl', 'rb') as file:
    scaler_versao_final = pickle.load(file)

In [ ]:
# Extrai os componentes do arquivo
modelo_carregado = modelo_versao_final["model"]
country_mapping = modelo_versao_final["country_mapping"]
education_mapping = modelo_versao_final["education_mapping"]
devtype_mapping  = modelo_versao_final['devtype_mapping']

In [ ]:
# Transform em novos dados
novos_dados_scaled = scaler_versao_final.transform(novos_dados)

In [ ]:
# Faz previsão com o modelo carregado do disco
y_pred = modelo_carregado.predict(novos_dados_scaled)

In [ ]:
print(y_pred)

In [ ]:
%watermark -a "Data Science Academy"

Retire os comentários das células abaixo e execute para checar as versões dos pacotes na sua máquina.

In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim